In [1]:
!pip install tensorflow numpy pandas tqdm scikit-learn segmentation-models tensorflow-addons flammkuchen opencv-python-headless imageio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import tensorflow as tf
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import numpy as np
import pandas as pd
import random
import cv2
import imageio as io
import flammkuchen as fl
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow  # Usar solo en Google Colab
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score

#from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.data import load_data, MAPE_V1, mape_apV1, mape_ppV1
from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.data import load_data, metric_mape, mape_ap, mape_pp
from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.Callbacks import get_callbacks
from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Models.GlottisNetV2_e import glottisnetV2_e


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Segmentation Models: using `tf.keras` framework.


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
def process_video_with_points(video_path, model_path, mask_save_path):

    if os.path.exists(video_path):
        # Load frames of video
        ims = io.mimread(video_path, memtest=False)

        # Load model
        model = load_model(model_path, compile=False)

        # Initialize lists to hold data
        masks, points, ims_orig = [], [], []

        # Iterate through frames
        for i in range(len(ims)):
            # Preprocess image for prediction
            img_orig = ims[i].astype(np.float32)
            img_orig = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img_orig, (256, 512))

            # Normalize and preprocess image
            normalizedImg = np.zeros(img.shape)
            img = cv2.normalize(img, normalizedImg, -1, 1, cv2.NORM_MINMAX)
            img = img[None, ..., None]

            # Prediction
            points_pred, seg_pred = model.predict(img)
            mask = np.asarray(np.squeeze(seg_pred))
            points_pred = np.asarray(np.squeeze(points_pred))

            # Process mask and points
            mask = np.round(mask)
            mask = cv2.resize(mask, (img_orig.shape[1], img_orig.shape[0]))
            mask = mask.astype(bool)


            # Append data to lists
            masks.append(mask)
            points.append(points_pred)
            ims_orig.append(img_orig)

        # Convert lists to numpy arrays and save
        masks = np.asarray(masks)
        points = np.asarray(points)

        fl.save(mask_save_path, {"mask": masks, 'files': ims_orig, "points": points}, compression='blosc')



In [5]:
def extract_points_from_heatmap(heatmap, original_size, resized_size):
    points = []
    for i in range(heatmap.shape[-1]):  # Asumimos dos puntos: anterior y posterior
        # Extrae las coordenadas de los puntos de máxima activación
        y, x = np.unravel_index(np.argmax(heatmap[..., i]), heatmap[..., i].shape)

        # Escala las coordenadas de vuelta al tamaño original de la imagen
        scale_x = original_size[1] / resized_size[1]
        scale_y = original_size[0] / resized_size[0]
        x = int(x * scale_x)
        y = int(y * scale_y)

        points.append((x, y))
    return points[0], points[1]
def draw_points_and_line(img, point1, point2, color_points=(0, 255, 255), color_line=(0, 255, 255), thickness=2):
    cv2.circle(img, point1, radius=5, color=color_points, thickness=-1)
    cv2.circle(img, point2, radius=5, color=color_points, thickness=-1)
    cv2.line(img, point1, point2, color=color_line, thickness=thickness)
    return img

def draw_contours_on_image(mask, img, contour_color=(255, 0, 0)):
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, contour_color, 2)
    return img

In [6]:

def recreate_video_with_segmentation(mask_save_path,output_video,fps,original_size):
    # Cargar datos guardados
    data = fl.load(mask_save_path)
    masks = data["mask"]
    images = data["files"]
    points_pred = data["points"]

    # Definir el codec y crear objeto VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video, fourcc, fps, (images[0].shape[1], images[0].shape[0]))

    # Procesar cada frame
    for i in range(len(images)):
        img_original = images[i].copy()

        if len(img_original.shape) == 2 or img_original.shape[2] == 1:
            img_original = cv2.cvtColor(img_original, cv2.COLOR_GRAY2BGR)

        img_original = img_original.astype(np.uint8)

        point1, point2 = extract_points_from_heatmap(points_pred[i], original_size = original_size, resized_size=(512, 256))
        img_bgr = cv2.cvtColor(img_original, cv2.COLOR_GRAY2BGR)
        img_with_points_and_line = draw_points_and_line(img_bgr, point1, point2)
        img_with_contours = draw_contours_on_image(masks[i], img_with_points_and_line)

        #img_with_points_and_line = draw_points_and_line(img_original, point1, point2)
        #img_with_contours = draw_contours_on_image(masks[i], img_with_points_and_line)

        img_with_contours = img_with_contours.astype(np.uint8)

        video_writer.write(img_with_contours)

    video_writer.release()



In [8]:

def predict_segmentation(video_path, model_path, mask_save_path,output_video, original_size):
  process_video_with_points(video_path, model_path, mask_save_path)
  cap = cv2.VideoCapture(video_path)
  fps = cap.get(cv2.CAP_PROP_FPS)
  cap.release()
  recreate_video_with_segmentation(mask_save_path,output_video,fps, original_size=original_size)

In [10]:
model_path = 'drive/MyDrive/Glottis/GlottisNetV2e/models/steps/epoch025.h5'  # Ajusta según tu path
video_path = 'drive/MyDrive/Glottis/videos_VPLAB/FN001.avi'  # Ajusta según tu path
mask_save_path = 'drive/MyDrive/Glottis/GlottisNetV2e/Masks/FN001.mask'  # Ajusta según tu path
output_video = '/content/drive/MyDrive/Glottis/videos_segmentados/FN001_seg.mp4'
predict_segmentation(video_path, model_path, mask_save_path,output_video, original_size=(250, 200))

1/1 [==============================] - 0s 487ms/step


In [ ]:
model_path = 'drive/MyDrive/Glottis/GlottisNetV2e/models/steps/epoch025.h5'  # Ajusta según tu path
video_path = 'drive/MyDrive/Glottis/videos_BAGLS/40.mp4'  # Ajusta según tu path
mask_save_path = 'drive/MyDrive/Glottis/GlottisNetV2e/Masks/40.mask'  # Ajusta según tu path
output_video = 'drive/MyDrive/Glottis/videos_segmentados/40_seg.mp4'
predict_segmentation(video_path, model_path, mask_save_path,output_video, original_size=(512, 256))

1/1 [==============================] - 0s 490ms/step


In [ ]:
model_path = 'drive/MyDrive/Glottis/GlottisNetV2e/models/steps/epoch025.h5'  # Ajusta según tu path
video_path = 'drive/MyDrive/Glottis/videos_VPLAB/FP007.avi'  # Ajusta según tu path
mask_save_path = 'drive/MyDrive/Glottis/GlottisNetV2e/Masks/FP007.mask'  # Ajusta según tu path
output_video = '/content/drive/MyDrive/Glottis/videos_segmentados/FP007_seg.mp4'
predict_segmentation(video_path, model_path, mask_save_path,output_video)